### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или иной способ оказался эффективнее остальных

In [2]:
import pandas as pd

In [4]:
news = pd.read_csv("./articles.csv")
print(news.shape)
news.sample(5)

(27000, 2)


,doc_id,title
10113,471122,США вновь сформировали управление 2-го флота в...
25500,488353,Слухи подтвердились — ученые впервые поймали г...
14622,478184,Традиционные скачки на льду финского озера Лев...
9723,470732,Глава Сирии Башар Асад не звонил президенту Ро...
20710,291129,Банк России второй раз в году повысил ключевую...


In [5]:
users = pd.read_csv("./users_articles.csv")
users.head(5)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"
3,u101138,"[5933, 6186, 5055, 6977, 5206, 488389]"
4,u108248,"[707, 1144, 2532, 2928, 3133, 324592]"


### 1. Получаем векторные представления новостей

In [9]:
from gensim.corpora.dictionary import Dictionary

In [10]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords

from razdel import tokenize

import pymorphy2

In [11]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

In [12]:
len(stopword_ru)

151

In [13]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [14]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
#   служебная функция, проверяет, принадлежность к str
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    
#   re.sub(A, b, str) заменяет все вхождения A в str на b
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    
    По сути это "нормализация" слов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [15]:
%%time
#Запускаем очистку текста.
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

C:\Users\Bu\AppData\Local\Temp\ipykernel_15112\1890200528.py:18: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: total: 27.2 s
Wall time: 27.2 s


In [16]:
%%time
#Запускаем лемматизацию текста.
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: total: 3min 18s
Wall time: 3min 18s


In [17]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [18]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: total: 2min 2s
Wall time: 45.5 s


In [19]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [20]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(10, 0.085120484),
 (15, 0.041348323),
 (16, 0.27557978),
 (20, 0.084210396),
 (22, 0.47132334),
 (24, 0.025502648)]

In [21]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: военный граница рейс сша российский высота польша
topic_1: мвд тело чёрный украина nn управление следствие
topic_2: который человек город произойти район обнаружить выяснить
topic_3: сон широко проба употребление чен уязвимый преобладать
topic_4: который это год проект россия страна новый
topic_5: ребёнок статья это который женщина газета год
topic_6: египет характерный бензин стена сдача спешить праздновать
topic_7: поверхность препарат вицепремьер годовой смерть британский год
topic_8: гражданин министерство медведев налог рубль курс офицер
topic_9: гражданин украинский памятник народный украина киев артист
topic_10: рейтинг место золото россиянин напротив третий золотой
topic_11: строительство турецкий турция объект аналог лесной живой
topic_12: земля пенсия болезнь тело человек доклад который
topic_13: это мочь который всё свой говорить человек
topic_14: год млн млрд рубль компания тыс цена
topic_15: энергия лётчик парка сочи звонок летний парк
topic_16: научный экипаж шта

In [22]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [23]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.037408,0.000000,0.048921,0.0,0.558018,0.347977,0.000000,0.000000,0.0,0.000000
1,4896,0.0,0.000000,0.0,0.0,0.000000,0.215529,0.0,0.000000,0.0,...,0.000000,0.000000,0.560709,0.0,0.076890,0.000000,0.125830,0.000000,0.0,0.000000
2,4897,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.041368,0.275478,0.000000,0.0,0.000000,0.084289,0.000000,0.471342,0.0,0.025505
3,4898,0.0,0.155627,0.0,0.0,0.115328,0.000000,0.0,0.000000,0.0,...,0.000000,0.033927,0.000000,0.0,0.000000,0.000000,0.062035,0.354480,0.0,0.000000
4,4899,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.041982,0.0,...,0.000000,0.035281,0.000000,0.0,0.431895,0.000000,0.064414,0.067994,0.0,0.262703


In [24]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [25]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [26]:
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.        , 0.04469708,
       0.12009995, 0.        , 0.04946413, 0.        , 0.        ,
       0.        , 0.        , 0.09232673, 0.        , 0.        ,
       0.07902366, 0.        , 0.02454727, 0.        , 0.27115631,
       0.26540291, 0.        , 0.04356829, 0.        , 0.        ])

In [28]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [29]:
get_user_embedding_mean(user_articles_list)

array([0.00993987, 0.03380553, 0.06850731, 0.        , 0.11444161,
       0.02985067, 0.        , 0.02452217, 0.        , 0.00237076,
       0.        , 0.        , 0.01130898, 0.06827939, 0.04269203,
       0.        , 0.00745547, 0.00939729, 0.09668492, 0.29069543,
       0.1187565 , 0.        , 0.05677145, 0.        , 0.00227495])

In [30]:
user_emb_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
user_emb_mean.columns = ['topic_{}'.format(i) for i in range(25)]
user_emb_mean['uid'] = users['uid'].values
user_emb_mean = user_emb_mean[['uid']+['topic_{}'.format(i) for i in range(25)]]


user_emb_median = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_emb_median.columns = ['topic_{}'.format(i) for i in range(25)]
user_emb_median['uid'] = users['uid'].values
user_emb_median = user_emb_median[['uid']+['topic_{}'.format(i) for i in range(25)]]


user_emb_max = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_emb_max.columns = ['topic_{}'.format(i) for i in range(25)]
user_emb_max['uid'] = users['uid'].values
user_emb_max = user_emb_max[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [6]:
target = pd.read_csv("./users_churn.csv")
# target.head(3)

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [33]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [34]:
X = pd.merge(user_emb_mean, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.000000,0.002674,0.052078,0.0,0.058276,0.118528,0.000000,0.008244,0.000000,...,0.000000,0.017687,0.040158,0.159601,0.236493,0.035108,0.047944,0.025952,0.000000,0
1,u108690,0.000000,0.040951,0.088498,0.0,0.112261,0.067257,0.000000,0.002375,0.009991,...,0.010881,0.006465,0.098227,0.182474,0.131068,0.000000,0.043076,0.000000,0.007035,1
2,u108339,0.004597,0.039313,0.202763,0.0,0.067911,0.040252,0.002488,0.010518,0.022813,...,0.009841,0.024982,0.033870,0.260805,0.128226,0.006666,0.012693,0.002110,0.009191,1


In [35]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [36]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [37]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.09584469, 0.04474776, 0.34722267, 0.1615007 , 0.05236467,
       0.0875165 , 0.4752641 , 0.04344313, 0.1408811 , 0.0866044 ])

In [39]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

param_mean = {
    'Param_title': 'mean',
    'Precision': precision[ix],
    'Recall': recall[ix],
    'F-Score': fscore[ix],
    'roc_auc_score': roc_auc_score(y_test, preds)
}

Best Threshold=0.222160, F-Score=0.639, Precision=0.539, Recall=0.784


In [40]:
param_mean

{'Param_title': 'mean',
 'Precision': 0.5393258426966292,
 'Recall': 0.7836734693877551,
 'F-Score': 0.6389351081530781,
 'roc_auc_score': 0.9324542124542124}

In [45]:
X = pd.merge(user_emb_median, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.0,0.000000,0.000000,0.0,0.022349,0.066577,0.0,0.0,0.000000,...,0.000000,0.0,0.014925,0.161918,0.211970,0.0,0.042994,0.0,0.0,0
1,u108690,0.0,0.021464,0.000000,0.0,0.057971,0.047648,0.0,0.0,0.000000,...,0.008172,0.0,0.055815,0.194081,0.138769,0.0,0.033347,0.0,0.0,1
2,u108339,0.0,0.034113,0.187633,0.0,0.059967,0.042737,0.0,0.0,0.013858,...,0.005228,0.0,0.028579,0.293301,0.158092,0.0,0.006967,0.0,0.0,1


In [46]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [47]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [48]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.07388933, 0.10293691, 0.46022034, 0.14598061, 0.04834257,
       0.04724772, 0.38804873, 0.0751936 , 0.14856929, 0.09365695])

In [49]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

param_median = {
    'Param_title': 'median',
    'Precision': precision[ix],
    'Recall': recall[ix],
    'F-Score': fscore[ix],
    'roc_auc_score': roc_auc_score(y_test, preds)
}

Best Threshold=0.293400, F-Score=0.740, Precision=0.721, Recall=0.759


In [50]:
param_median

{'Param_title': 'median',
 'Precision': 0.7209302325581395,
 'Recall': 0.7591836734693878,
 'F-Score': 0.7395626242544732,
 'roc_auc_score': 0.9647351590208731}

In [51]:
X = pd.merge(user_emb_max, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.000000,0.016043,0.269720,0.0,0.189566,0.457917,0.000000,0.049464,0.000000,...,0.000000,0.081577,0.146104,0.362613,0.605170,0.152096,0.109602,0.082084,0.000000,0
1,u108690,0.000000,0.162623,0.330768,0.0,0.443540,0.159816,0.000000,0.014248,0.059946,...,0.024888,0.024726,0.282086,0.260931,0.186881,0.000000,0.108757,0.000000,0.022902,1
2,u108339,0.027581,0.096189,0.395155,0.0,0.161005,0.098088,0.014928,0.063108,0.067123,...,0.037692,0.093150,0.081117,0.430386,0.231341,0.026978,0.034864,0.012662,0.055149,1


In [52]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [53]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [54]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.04000647, 0.00161775, 0.82376034, 0.05078062, 0.0172771 ,
       0.00244198, 0.05641271, 0.00879557, 0.0790501 , 0.08372688])

In [55]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

param_max = {
    'Param_title': 'max',
    'Precision': precision[ix],
    'Recall': recall[ix],
    'F-Score': fscore[ix],
    'roc_auc_score': roc_auc_score(y_test, preds)
}

Best Threshold=0.338618, F-Score=0.771, Precision=0.759, Recall=0.784


In [56]:
param_max

{'Param_title': 'max',
 'Precision': 0.758893280632411,
 'Recall': 0.7836734693877551,
 'F-Score': 0.7710843373493975,
 'roc_auc_score': 0.9691563462992036}

In [57]:
df_param = pd.DataFrame()
df_param = df_param.append(param_mean, ignore_index=True)
df_param = df_param.append(param_median, ignore_index=True)
df_param = df_param.append(param_max, ignore_index=True)
df_param

C:\Users\Bu\AppData\Local\Temp\ipykernel_15112\1074978597.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_param = df_param.append(param_mean, ignore_index=True)
C:\Users\Bu\AppData\Local\Temp\ipykernel_15112\1074978597.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_param = df_param.append(param_median, ignore_index=True)
C:\Users\Bu\AppData\Local\Temp\ipykernel_15112\1074978597.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_param = df_param.append(param_max, ignore_index=True)


,Param_title,Precision,Recall,F-Score,roc_auc_score
0,mean,0.539326,0.783673,0.638935,0.932454
1,median,0.720930,0.759184,0.739563,0.964735
2,max,0.758893,0.783673,0.771084,0.969156
